In [1]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v2/codes


In [2]:
cd ..

/Users/satouwataru/Desktop/DiveIntoCode/git/diveintocode-ml/GraduationWork/v2


In [19]:
import numpy as np
import pandas as pd
import optuna.integration.lightgbm as lgb_o
from sklearn.metrics import mean_squared_error

import preprocessing
from right_gbm import my_model as my_right_gbm
from sklearn.model_selection import train_test_split

def pred_toto(toto_n, toto_kind, pred_par_day = False):

    df_toto_preds = get_y_preds(toto_n, toto_kind, pred_par_day)
    yyyyMMdd = df_toto_preds.min(axis=0)['開催日']

    if pred_par_day:
        df_y_preds = pd.read_csv('data/model/y_preds_par_day.csv', index_col=0).reset_index(drop = True)
    else:
        df_y_preds = pd.read_csv('data/model/y_preds_par_year.csv', index_col=0).reset_index(drop = True)
    df_y_preds = df_y_preds[df_y_preds['開催日'] < yyyyMMdd]

    df_toto = pd.read_csv('data/marge/toto_info.csv', index_col=0).reset_index(drop = True)
    df_toto = df_toto[['第n回', '種別', '開催日', 'ホーム', 'アウェイ', 'くじ結果'] ]
    df_toto = df_toto[df_toto['くじ結果'] != '102'].reset_index(drop = True)

    drop_columns = ['第n回', '種別','is_even_1_1_lgbm', 'is_lose_1_1_lgbm', 'w_e_l_2_2_lgbm' ]
    train = pd.merge(df_y_preds, df_toto).drop(columns = drop_columns)
    test = df_toto_preds.drop(columns = drop_columns)

    train = preprocessing.team_to_index(train, 'ホーム')
    train = preprocessing.team_to_index(train, 'アウェイ')
    test = preprocessing.team_to_index(test, 'ホーム')
    test = preprocessing.team_to_index(test, 'アウェイ')

    category_columns = ['ホーム', 'アウェイ']
    for column in category_columns:
        train[column] = pd.Series(train[column]).astype('category')
        test[column] = pd.Series(test[column]).astype('category')

    train, val = train_test_split(train, train_size=0.8)

    x_train = train.drop(columns = ['くじ結果'])
    y_train = train['くじ結果'].values
    x_val = val.drop(columns = ['くじ結果'])
    y_val = val['くじ結果'].values
    x_test = test
#     return x_train, x_val, y_train, y_val
    model = model_toto()
    model.fit(x_train, x_val, y_train, y_val)
    y_pred, y_pred_proba = model.predict(x_test)

    df_pred = pd.DataFrame(y_pred_proba, columns=['分', '勝', '敗'])
    df_toto = x_test[['開催日', 'ホーム', 'アウェイ']]
    df_result = pd.concat([df_pred, df_toto], axis=1)
    df_result = preprocessing.index_to_team(df_result, 'ホーム')
    df_result = preprocessing.index_to_team(df_result, 'アウェイ')

    df_result = df_result.reindex(columns=['開催日', 'ホーム', 'アウェイ', '勝', '敗', '分'])

    return df_result

def get_y_preds(toto_n, toto_kind, pred_par_day = False):
    
    if pred_par_day:
    
        # 前処理
        preprocessing.preprocessing(toto_n, toto_kind, pred_par_day)

        # 予測値を算出
        df_result, df_acuracy = my_right_gbm.predict_toto()
        df_result = preprocessing.index_to_team(df_result, 'H_Team')
        df_result = preprocessing.index_to_team(df_result, 'A_Team')
        df_result['toto_n'] = toto_n
        df_result['toto_kind'] = toto_kind

        # 結果を整形
        df_y_preds = pd.read_csv('data/model/y_preds_par_day.csv', index_col=0).reset_index(drop = True)
        df_toto = pd.read_csv('data/marge/toto_info.csv', index_col=0).reset_index(drop = True)
        df_toto = df_toto[['第n回', '種別', '開催日', 'ホーム', 'アウェイ']]

        df_marge = pd.merge(df_toto, df_result, 
                     left_on=['第n回', '種別', 'ホーム'],
                     right_on=['toto_n', 'toto_kind', 'H_Team']).drop(columns = ['H_Team', 'A_Team', '年月日', 'toto_n', 'toto_kind'])

        df_y_preds = pd.concat([df_y_preds, df_marge]).reset_index(drop = True)
        df_y_preds.to_csv("data/model/y_preds_par_day.csv")
        
        return df_marge
    else:
        # 前処理
        df = preprocessing.preprocessing(toto_n, toto_kind, pred_par_day)
        yyyyMMdd = df[df['train_test'] == 'toto'].iloc[:1]['年月日'].values[0]
        year = str(yyyyMMdd)[:4]
        # 予測値を算出
        df_result, df_acuracy = my_right_gbm.predict_toto(year)
        df_result = preprocessing.index_to_team(df_result, 'H_Team')
        df_result = preprocessing.index_to_team(df_result, 'A_Team')
        df_result['toto_n'] = toto_n
        df_result['toto_kind'] = toto_kind

        # 結果を整形
        df_y_preds = pd.read_csv('data/model/y_preds_par_year.csv', index_col=0).reset_index(drop = True)
        df_toto = pd.read_csv('data/marge/toto_info.csv', index_col=0).reset_index(drop = True)
        df_toto = df_toto[['第n回', '種別', '開催日', 'ホーム', 'アウェイ']]

        df_marge = pd.merge(df_toto, df_result, 
                     left_on=['第n回', '種別', 'ホーム'],
                     right_on=['toto_n', 'toto_kind', 'H_Team']).drop(columns = ['H_Team', 'A_Team', '年月日', 'toto_n', 'toto_kind'])

        df_y_preds = pd.concat([df_y_preds, df_marge]).reset_index(drop = True)
        df_y_preds.to_csv("data/model/y_preds_par_year.csv")
        
        return df_marge
    
class model_toto:
    
    def fit(self, x_train, x_val, y_train, y_val):
        
        lgb_train = lgb_o.Dataset(x_train, y_train)
        lgb_eval = lgb_o.Dataset(x_val, y_val) 
        # 学習用パラメータ
        lgbm_params = {
            'objective': 'multiclass',
            'metric': 'multi_logloss',
            'num_class': 3,
            'verbosity': -1
        }
        # 学習
        model = lgb_o.train(lgbm_params,
                        lgb_train,
                        valid_sets=lgb_eval,
                        early_stopping_rounds=100,
                        verbose_eval=200,)
        self.model = model

        y_pred_proba = self.model.predict(x_val, num_iteration=self.model.best_iteration)
        y_pred = np.argmax(y_pred_proba, axis=1)

        # Accuracy の計算
        accuracy = sum(y_val == y_pred) / len(y_val)
        print('accuracy:', accuracy)
        
        return y_pred, y_pred_proba, accuracy
    
    def predict(self, x_test):
        y_pred_proba = self.model.predict(x_test, num_iteration=self.model.best_iteration)
        y_pred = np.argmax(y_pred_proba, axis=1)
        return y_pred, y_pred_proba

In [21]:
toto_n, toto_kind = 1238,'toto'
df_toto_preds = get_y_preds(toto_n, toto_kind, False)

<ipython-input-21-575273e040f4>:2: DtypeWarning: Columns (12,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df_toto_preds = get_y_preds(toto_n, toto_kind, False)


accuracy: 0.5379746835443038
accuracy: 0.5063291139240507
accuracy: 0.5
rmse :  1.5474082068042723
rmse :  1.1572090569991256
rmse :  1.0777432864266587


In [23]:
# 前処理
df = preprocessing.preprocessing(toto_n, toto_kind, False)
yyyyMMdd = df[df['train_test'] == 'toto'].iloc[:1]['年月日'].values[0]
year = str(yyyyMMdd)[:4]
# 予測値を算出
df_result, df_acuracy = my_right_gbm.predict_toto(year)

/Users/satouwataru/.pyenv/versions/anaconda3-2020.07/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (12,22) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


accuracy: 0.5253164556962026
accuracy: 0.45569620253164556
accuracy: 0.3987341772151899
rmse :  1.3753556833117502
rmse :  1.0660960401194293
rmse :  1.0250273237533185


In [27]:
df[df['train_test'] == 'toto']

,年月日,節,スタジアム,H_勝点,H_試合,H_勝,H_分,H_敗,H_得点,H_失点,...,H_MF,H_FW,A_DF,A_MF,A_FW,H_順位,A_順位,H_Team,A_Team,train_test
6320,20210505.0,12.0,101.0,15.0,11.0,4.0,3.0,4.0,8.0,7.0,...,5.0,2.0,4.0,4.0,2.0,9.0,6.0,36.0,28.0,toto
6321,20210505.0,12.0,7.0,12.0,11.0,3.0,3.0,5.0,8.0,11.0,...,3.0,3.0,4.0,4.0,2.0,15.0,16.0,37.0,26.0,toto
6322,20210505.0,12.0,20.0,9.0,11.0,2.0,3.0,6.0,12.0,18.0,...,4.0,2.0,4.0,4.0,2.0,21.0,4.0,38.0,34.0,toto
6323,20210505.0,12.0,14.0,13.0,11.0,3.0,4.0,4.0,12.0,17.0,...,5.0,2.0,4.0,4.0,2.0,14.0,19.0,31.0,29.0,toto
6324,20210505.0,12.0,17.0,14.0,11.0,4.0,2.0,5.0,12.0,18.0,...,6.0,1.0,4.0,4.0,2.0,11.0,10.0,40.0,20.0,toto


0        19970308
1        19970315
2        19970319
3        19970322
4        19970326
           ...   
22632    20181202
22633    20190721
22634    20191027
22635    20200705
22636    20201025
Name: 年月日, Length: 22637, dtype: object